### 交叉熵 
#### 刻画了两个概率密度分布之间的距离，它是分类问题中使用比较广的一种损失函数

给定两个概率分布$p$和$q$，通过$q$来表示$p$的交叉熵为：$$H(p,q)=\sum_{x}p(x)logq(x)$$
交叉熵刻画的是两上概率分布之间的距离；
概率分布刻画了不同事件发生的概率。当事件总数有限的情况下，概率分布函数 $p(X=x)$ 满足：
$$\forall p(X=x)\in[0,1] 且 \sum_{x}p(X=x)=1$$
即任何事件发生的概率都在0和1之间，且总有某一个事件发生（概率的和为1）

### 然而神经网络的输出却不一定是一个概率分布（即不总在0-1之间）
#### Softmax回归是一层额外的处理层，将神经网络的输出变成一个概率分布；

假设 原始的神经网络输出为$y_1,y_2,\cdots,y_n$，那么，经过Softmax回归处理后的输出为：$$softmax(y)_{i}=y_{i}^{'}=\frac{e^{yi}}{\sum_{j=1}^{n}e^{yj}}$$

### Tensorflow 中的交叉熵 tf.nn.sigmoid_cross_entropy_with_logits  使用了softmax回归之后的交叉熵 $$loss_{ij}=-[y_{ij}*lnp_{ij}+(1-y_{ij}ln(1-p_{ij})]$$

当交叉熵作为神经网络的损失函数时，$p$代表正确答案，$q$代表预测值；

In [65]:
import numpy as np

def cross_entropy(p,q):
    llen=len(p)
#     return -1*sum(p[i]*np.log10(q[i]+(1-p[i])*np.log10(1-q[i])) for i in range(llen))
    return -1*sum(p[i]*np.log(q[i]) for i in range(llen))

a=[1,0,0]
b=[0.5,0.4,0.1]
c=[0.8,0.1,0.1]
print(cross_entropy(a,b)/3)
print(cross_entropy(a,c)/3)

0.23104906018664842
0.07438118377140324


### 最常用的损失函数：均方误差（MSE,mean squared error) $$MSE(y,y^{'})=\frac{\sum_{i=1}^{n}(y_i-y_{i}^{'})^2}{n}$$
$ mse = tf.reduce\_mean(tf.square(y^{'}-y))$ 

### Tensorflow 自定义损失函数 例：$$ Loss(y,y^{'}=\sum_{i=1}^{n}f(y_i,y^{'}),\, f(x,y)=\left\{
\begin{aligned}
a(x-y) &   & x>y \\ 
b(y-x) &   & x\leq y 
\end{aligned} \right.
$$

Tensorflow 实现： $loss = tf.reduce\_sum(tf.where(tf.greater(v1,v2),(v1-v2)*a,(v2-v1)*b))$

In [1]:
import tensorflow as tf 
from numpy.random import RandomState

batch_size=8

# 两个输入节点
x=tf.placeholder(tf.float32,shape=(None,2), name='x-input')

# 回归问题一般只有一个输出节点
y_=tf.placeholder( tf.float32, shape=(None,1), name='y-input')

# 定义了一个单层的神经网络前向传播的过程，这里就是简单加权和
w1=tf.Variable(tf.random_normal( [2,1], stddev=1, seed=1))
y=tf.matmul(x,w1)

# 定义预测多了和预测少了的成本
loss_less=10
loss_more=1
 
loss=tf.reduce_sum(tf.where(tf.greater(y,y_), (y-y_)*loss_more, (y_-y)*loss_less))
train_step=tf.train.AdadeltaOptimizer(0.001).minimize(loss)

# 通过随机数生成一个模拟数据集
rdm=RandomState(1)
dataset_size=128

X=rdm.rand(dataset_size,2)

# -0.05~0.05之间的随机数
Y=[[x1+x2+rdm.rand()/10.0 - 0.05] for (x1,x2) in X]

with tf.Session() as sess:
    init_op=tf.global_variables_initializer()
    sess.run(init_op)
    
    STEPS=5000
    for i in range(STEPS):
        start=(i*batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        sess.run(train_step, feed_dict={x:X[start:end],y_:Y[start:end]})

        print(sess.run(w1))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

### 正则化
正则化的思想就是在损失函数中加入刻画模型复杂度的指标，用于避免过拟合问题。

优化$$J(\theta) + \lambda R(w)$$ 其中，$J(\theta)$为损失函数，$R(w)$ 刻画了模型的复杂度

### L1正则化 
$$R(w)=\parallel w \parallel _{1} = \sum_{i}\mid w_i \mid$$
### L2正则化 
$$R(w)=\parallel w \parallel _{2}^{2} = \sum_{i}\mid w_{i}^2 \mid$$

L1正则化会让参数变得更稀疏，即便更多的参数变为0，达到类似特征选择的功能。L2不会

****
****
****

## **机器学习中常用的损失函数**

https://blog.csdn.net/google19890102/article/details/50522945
***

* ### **0-1损失函数**

##### $L_{01}(m) = \left\{\begin{align} 0,\quad if \quad m\geq 0 \\ 1,\quad if \quad m<0 \end{align}\right.$  ,$\quad\quad$等价于  $\frac{1}{2}(1-sign(m))$


* ### **Log损失函数**

Log损失是0-1损失函数的一种代理函数。log损失的具体形式：<font size=3>$log(1 + exp(-m))$</font> 。 运用log损失的典型分类器是**Logistic回归算法**

* ### **Hinge损失函数**

Hinge损失是0-1损失函数的一种代理函数。Hinge损失的具体形式：<font size=3>$ max(0, 1-m )$</font> 。 运用Hinge损失的典型分类器是**SVM 算法**

* ### **指数损失函数**

指数损失是0-1损失函数的一种代理函数。指数损失的具体形式：<font size=3>$ exp(-m) $</font> 。 运用指数损失的典型分类器是**AdaBoost 算法**

* ### **感知损失函数**

感知损失是Hinge损失的一个变种，感知损失的具体形式：<font size=3>$ max(0, -m )$</font> 。 运用感知损失的典型分类器是**感知机算法**